In [7]:

!pip install -U boto3 langchain langchain_aws -U -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/07/0f/aa5b3ca9fa4bbb7db3e9150cc16269867ebbf245b67400aaba97ebbfede9/boto3-1.34.104-py3-none-any.whl (139 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/4b/28/da40a6b12e7842a0c8b443f8cc5c6f59e49d7a9071cfad064b9639c6b044/langchain-0.1.20-py3-none-any.whl (1.0 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/87/e0/57d1e6e2db78129b6148e5af00bb4d22bcd3d38af3eed802afc78355e5e3/langchain_aws-0.1.3-py3-none-any.whl (37 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/aa/e0/7ec1a84a66c330fcc5a0c8604a12c5bd34802f5f6f8eda501cd3210767a8/botocore-1.34.104-py3-none-any.whl (12.2 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/1b/d3/1f4d1941ae5a627299c8ea052847b99ad6674b97b699d8a08fc4faf25d3e/langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/43/8b/48b7e6de9041d2b33d5108e

In [1]:
import boto3

In [2]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage,AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
boto3_bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name='us-east-1'
)

In [4]:
llm_sonnet = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                client = boto3_bedrock,
                  model_kwargs={"temperature": 0,
                                "top_k":10,
                                "max_tokens": 1024,
                                "top_p":0.5,
                                "stop_sequences":['</standalone_question>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

In [5]:
llm_llama = ChatBedrock(model_id="meta.llama3-70b-instruct-v1:0",
                  model_kwargs={"temperature": 0.5,
                                "max_gen_len": 1024,
                                "top_p":0.9,
                               },
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )

In [6]:
prompt = "Given the following conversation in <conversation></conversation>, and a follow up user question in <question></question>.\\n<conversation>\\nuser: aws clean room是什么？\\nAI: AWS Clean Rooms是一种安全的多方计算服务,允许不同组织在不共享原始数据的情况下,对彼此的数据进行联合分析。它采用差分隐私技术,确保数据隐私和安全。与其他类似服务相比,AWS Clean Rooms无需将数据移动到其他平台,可直接在AWS S3上进行分析。它还与AWS Data Exchange集成,方便寻找数据合作伙伴。AWS Clean Rooms将取代亚马逊广告的AMC服务,为更广泛的行业提供数据合作解决方案。\\n</conversation>\\n\\n<question>\\n他有什么优势？\\n</question>\\n\\nplease use the context in the chat conversation to rephrase the user question to be a standalone question, respond in the original language of user's question, don't translate the chat history and user question.\\nif you don't understand the user's question, or the question is not relevant to the conversation. please keep the orginal question.\\nSkip the preamble, don't explain, go straight into the answer. Please put the standalone question in <standalone_question> tag\\n"

In [7]:
llm_llama.invoke(prompt)

 

<standalone_question>什么是AWS Clean Rooms的优势？</standalone_question>

它无需将数据移动到其他平台,可直接在AWS S3上进行分析。它还与AWS Data Exchange集成,方便寻找数据合作伙伴。

AIMessage(content=' \n\n<standalone_question>什么是AWS Clean Rooms的优势？</standalone_question>\n\n它无需将数据移动到其他平台,可直接在AWS S3上进行分析。它还与AWS Data Exchange集成,方便寻找数据合作伙伴。', additional_kwargs={'usage': {}}, response_metadata={'model_id': 'meta.llama3-70b-instruct-v1:0'}, id='run-dde4fa81-601e-4128-b295-dabcf6250f0d-0')

In [15]:
template = """
You are a helpful assistant
Please answer users' question as possible as you can:
User question: {question}
"""


In [16]:
output_parser = StrOutputParser()
prompt_template = ChatPromptTemplate.from_template(template)

In [17]:
chain = prompt_template | llm_sonnet |output_parser

In [18]:
question = "who is the director of Terminator 2?"

In [19]:
chain.invoke({"question":question})

The director of Terminator 2: Judgment Day is James Cameron.

Some additional details:

- Terminator 2 was released in 1991 and was a sequel to the 1984 film The Terminator, which was also directed by James Cameron.

- It starred Arnold Schwarzenegger as the Terminator character, with Linda Hamilton and Edward Furlong in the other main roles.

- The film was a massive critical and commercial success, earning over $500 million at the box office against a $100 million budget.

- It was groundbreaking for its special effects, which still hold up incredibly well today thanks to the pioneering use of computer-generated imagery (CGI).

- James Cameron co-wrote the screenplay along with William Wisher Jr. He is considered a master of the sci-fi and action genres.

- Other famous films directed by James Cameron include Aliens, Titanic, and Avatar.

So in summary, the acclaimed director who brought the iconic Terminator 2: Judgment Day to life was the visionary filmmaker James Cameron.

'The director of Terminator 2: Judgment Day is James Cameron.\n\nSome additional details:\n\n- Terminator 2 was released in 1991 and was a sequel to the 1984 film The Terminator, which was also directed by James Cameron.\n\n- It starred Arnold Schwarzenegger as the Terminator character, with Linda Hamilton and Edward Furlong in the other main roles.\n\n- The film was a massive critical and commercial success, earning over $500 million at the box office against a $100 million budget.\n\n- It was groundbreaking for its special effects, which still hold up incredibly well today thanks to the pioneering use of computer-generated imagery (CGI).\n\n- James Cameron co-wrote the screenplay along with William Wisher Jr. He is considered a master of the sci-fi and action genres.\n\n- Other famous films directed by James Cameron include Aliens, Titanic, and Avatar.\n\nSo in summary, the acclaimed director who brought the iconic Terminator 2: Judgment Day to life was the visionary filmmaker James 